## Load Data Set

In [1]:
import zipfile
import os
import numpy as np
import pathlib
import pandas as pd
from math import ceil
import tensorflow as tf
import numpy as np
import IPython.display as display
import keras
from keras import backend as K
from matplotlib import pyplot as plt
from keras.utils import to_categorical


print(tf.__version__)

root_path = './'
dataset_path = os.path.join(root_path, 'tf_dataset_gray')

models_path = os.path.join(root_path, 'saved_models_gray')
if not os.path.exists(models_path):
  os.mkdir(models_path)

Using TensorFlow backend.


1.13.1


In [2]:
BATCH_SIZE = 128


image_feature_description = {
    'enc': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
    'img': tf.io.FixedLenSequenceFeature([], tf.string, allow_missing=True)
}


def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)


raw_train_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path,'train.tfrecords'))
parsed_train_dataset = raw_train_dataset.map(_parse_image_function)

# raw_val_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path, 'val.tfrecords'))
# parsed_val_dataset = raw_val_dataset.map(_parse_image_function)

# raw_test_dataset = tf.data.TFRecordDataset(os.path.join(dataset_path, 'test.tfrecords'))
# parsed_test_dataset = raw_test_dataset.map(_parse_image_function)


parsed_train_dataset = parsed_train_dataset.repeat()
parsed_train_dataset = parsed_train_dataset.shuffle(6000)
parsed_train_dataset = parsed_train_dataset.batch(BATCH_SIZE)
dataset_iterator = parsed_train_dataset.make_one_shot_iterator()

variable_dataset = dataset_iterator.get_next()

## Wasserstein GAN

In [3]:
def d_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

## Model Design

In [4]:
enc_len = 0
age_len = 0
img_shape = (32, 32, 1)
width, height, depth = (32, 32, 1)
img_len = np.prod(img_shape)
latent_dim = enc_len + age_len + img_len
noise_len = 100
input_dim = enc_len + age_len + noise_len
cond_len = enc_len + age_len


def build_discriminator():
    conv = keras.Sequential([
        # conv block 1
        keras.layers.Conv2D(
            filters=16,
            kernel_size=(3, 3),
            strides=2,
            input_shape=img_shape
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        keras.layers.BatchNormalization(),

        # conv block 2
        keras.layers.Conv2D(
            filters=32,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
        keras.layers.BatchNormalization(),
        
        # conv block 3
        keras.layers.Conv2D(
            filters=3,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.leaky_relu),
    ])
    
    conv.summary()
    
    model = keras.Sequential([
        # dense 1
        keras.layers.Dense(128, input_shape=(27,)),
        keras.layers.Activation(tf.nn.relu),
        
        # output
        keras.layers.Dense(1),
        keras.layers.Activation(tf.nn.sigmoid),
    ])
    
#     clf = keras.Sequential([
#         # dense 1
#         keras.layers.Dense(128, input_shape=(27+128+8,)),
#         keras.layers.Activation(tf.nn.relu),
        
#         # output
#         keras.layers.Dense(age_len),
#         keras.layers.Activation(tf.nn.softmax),
#     ])
    
    model.summary()
    
#     # condition
#     c1 = keras.layers.Input(shape=(enc_len,))
#     c2 = keras.layers.Input(shape=(age_len,))
    
    # image
    z = keras.layers.Input(shape=img_shape)
    
    # convolution
    zout = conv(z)
    
    # flatten image
    z_flat = keras.layers.Flatten()(zout)
    
    # concatenation
#     inputs = keras.layers.concatenate([z_flat])
    
    # real or fake
    outputs = model(z_flat)
    
    # age label
#     classes = clf(inputs)
    
    
    return keras.models.Model(z, outputs)


def build_generator():
    
    conv = keras.Sequential([
        # transpose conv block 1
        keras.layers.Conv2DTranspose(
            filters=16,
            kernel_size=(3, 3),
            strides=1,
            input_shape=(11, 11, 1)
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 2
        keras.layers.Conv2DTranspose(
            filters=32,
            kernel_size=(3, 3),
            strides=2
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),
        
        # transpose conv block 3
        keras.layers.Conv2DTranspose(
            filters=32,
            kernel_size=(2, 2),
            strides=1
        ),
        keras.layers.Activation(tf.nn.relu),
        keras.layers.BatchNormalization(),

        # transpose conv block 4
        keras.layers.Conv2DTranspose(
            filters=1,
            kernel_size=(5, 5),
            strides=1
        ),
        
        # output
        keras.layers.Activation(tf.nn.tanh)
    ])
    
    conv.summary()
    
    model = keras.Sequential([
        # dense 1
        keras.layers.Dense(121, input_shape=(input_dim,)),
        keras.layers.Activation(tf.nn.relu),
        
        # reshape 1d to 3d
        keras.layers.Reshape((11, 11, 1))
    ])
    
    model.summary()
    
#     # condition
#     c1 = keras.layers.Input(shape=(enc_len,))
#     c2 = keras.layers.Input(shape=(age_len,))
    
    # noise
    x = keras.layers.Input(shape=(noise_len,))

    # concatenation
#     inputs = keras.layers.concatenate([x])
    
    # flat dense output
    out_1 = model(x)
    
    # transpose conv output
    outputs = conv(out_1)
    
    return keras.models.Model(x, outputs)


discriminator = build_discriminator()
generator = build_generator()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 15, 15, 16)        160       
_________________________________________________________________
activation_1 (Activation)    (None, 15, 15, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 15, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
activation_2 (Activation)    (None, 7, 7, 32)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 7, 7, 32)          128       
_________________________________________________________________
conv

In [5]:
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
sequential_4 (Sequential)    (None, 11, 11, 1)         12221     
_________________________________________________________________
sequential_3 (Sequential)    (None, 32, 32, 1)         10049     
Total params: 22,270
Trainable params: 22,110
Non-trainable params: 160
_________________________________________________________________


In [6]:
discriminator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 1)         0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 3, 3, 3)           5859      
_________________________________________________________________
flatten_1 (Flatten)          (None, 27)                0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 1)                 3713      
Total params: 9,572
Trainable params: 9,476
Non-trainable params: 96
_________________________________________________________________


## Generative Adversarial Network

In [7]:
GLR = 0.01  # generator
DLR = 0.01  # discriminator


discriminator.compile(
    optimizer=keras.optimizers.Adam(DLR, 0.5),
    loss=d_loss,
    metrics=['accuracy']
)


# # condition
# c1 = keras.layers.Input(shape=(enc_len,))
# c2 = keras.layers.Input(shape=(age_len,))

# noise
x = keras.layers.Input(shape=(noise_len,))

# freeze discriminator
discriminator.trainable = False

# output
z = generator(x)
out = discriminator(z)

# GAN
gan = keras.models.Model(inputs=x, outputs=out)

gan.compile(
    optimizer=keras.optimizers.Adam(GLR , 0.5),
    loss=d_loss,
    metrics=['accuracy'])

In [8]:
gan.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 100)               0         
_________________________________________________________________
model_2 (Model)              (None, 32, 32, 1)         22270     
_________________________________________________________________
model_1 (Model)              (None, 1)                 9572      
Total params: 31,842
Trainable params: 22,110
Non-trainable params: 9,732
_________________________________________________________________


## Visualization Method

In [9]:
# from google.colab import drive
import os


# drive.mount('/content/gdrive', force_remount=True)

root_path = './'
tgt_pth = os.path.join(root_path, 'visualize_age-cgan-v6')

if not os.path.exists(tgt_pth):
  os.mkdir(tgt_pth)

In [13]:
def visualizeGAN(e, z_real, z_fake):

    fig, axes = plt.subplots(32, 8, figsize=(80, 72))

    r_real = 0
    r_fake = 0
    for row, axe in enumerate(axes):
        for col, cell in enumerate(axe):
            if row % 2 == 0:
                cell.imshow(np.squeeze(z_real[r_real * 8 + col], axis=-1), cmap='gray')
            else:
                cell.imshow(
                    np.squeeze(
                        (z_fake[r_fake * 8 + col] * 255).astype(np.uint8),
                        axis=-1
                    ),
                    cmap='gray'
                )

            cell.axis("off")

        if row % 2 == 0:
            r_real += 1
        else:
            r_fake += 1

    plt.axis("off")
    
    fig.savefig(os.path.join(tgt_pth, '{}.jpg'.format(str(e).zfill(3))))
    
    plt.close()

## Load Batch

In [14]:
def load_noise():
    
    y_real = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_real = y_real.eval()

        while True:
            values = sess.run([variable_dataset])
            row = values[0]

            sz = row['img'].shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
            # concatenate face + age + noise
#             c1 = row['enc']
#             c2 = tf.cast(row['age'], tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            
            yield x, y_real


def load_batch():
    
    y_fake = tf.zeros((BATCH_SIZE,))
    y_real = tf.ones((BATCH_SIZE,))
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.per_process_gpu_memory_fraction = 0.40

    with tf.Session(config=config) as sess:
        tf.initialize_all_variables().run()
        
        # run once
        y_fake = y_fake.eval()
        y_real = y_real.eval()

        while True:
            values = sess.run([variable_dataset])
            row = values[0]

            sz = row['img'].shape[0]

            if sz != BATCH_SIZE:
                continue
            
            # fake data
#             c1 = row['enc']
#             c2 = tf.cast(row['age'], tf.float32).eval()
            x = tf.random.normal((sz, noise_len,)).eval()
            z_fake = generator.predict(x)

            # real data
#             c1 = row['enc']
#             c2 = tf.cast(row['age'], tf.float32).eval()
            z_real = tf.reshape(tf.io.decode_raw(row['img'], tf.int64), (-1, width, height, depth)).eval()
                        
            yield x, z_fake, y_fake, z_real, y_real

## Train Model

In [15]:
EPOCHS = 10000
STEPS = 1  # 60000 // BATCH_SIZE


train_loss_g = []
train_loss_d = []

train_acc_g = []
train_acc_d = []


disc_itr = load_batch()
gen_itr = load_noise()


# epochs
for e in range(EPOCHS):

    #batches
    loss = []
    acc = []

    for p in range(STEPS * 4):
        
        x, z_fake, y_fake, z_real, y_real = next(disc_itr)

        # train
        loss_2, acc_2 = discriminator.train_on_batch(z_real, y_real)
        loss_1, acc_1 = discriminator.train_on_batch(z_fake, y_fake)

        batch_loss = 0.5 * (loss_1 + loss_2)
        batch_acc = 0.5 * (acc_1 + acc_2)

        loss.append(batch_loss)
        acc.append(batch_acc)

    train_loss_d.append(np.mean(np.array(loss)))
    train_acc_d.append(np.mean(np.array(acc)))

    #batches
    loss = []
    acc = []

    for p in range(STEPS):

        x, y_true = next(gen_itr)

        # train
        loss_1, acc_1 = gan.train_on_batch(x, y_true)

        loss.append(loss_1)
        acc.append(acc_1)

    train_loss_g.append(np.mean(np.array(loss)))
    train_acc_g.append(np.mean(np.array(acc)))


    print("Epoch: {}, Steps: {}, Discriminator Accuracy: %{:.2f}, GAN Accuracy: %{:.2f}".format(
          e,
          STEPS,
          train_acc_d[-1] * 100,
          train_acc_g[-1] * 100
      ))

    if e % 25 == 0:
        ## visualize results
        x, z_fake, y_fake, z_real, y_real = next(disc_itr)
        visualizeGAN(e, z_real, z_fake)
        
        ## save model
        pth = os.path.join(models_path, 'gan.h5')
        gan.save(pth)

        pth = os.path.join(models_path, 'generator-{}-{}-{}.h5'.format(e, train_loss_g[-1], train_acc_g[-1]))
        generator.save(pth)

        pth = os.path.join(models_path, 'discriminator.h5')
        discriminator.save(pth)

Exception ignored in: <generator object load_batch at 0x7f8926cb48e0>
Traceback (most recent call last):
  File "<ipython-input-11-34d66c71049c>", line 73, in load_batch
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1573, in __exit__
    exec_tb)
  File "/usr/lib/python3.6/contextlib.py", line 99, in __exit__
    self.gen.throw(type, value, traceback)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/ops.py", line 5069, in get_controller
    type(default))
AssertionError: Nesting violated for default stack of <class 'tensorflow.python.client.session.Session'> objects
Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x7f89269b2e48>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "/usr/

Epoch: 0, Steps: 1, Discriminator Accuracy: %51.17, GAN Accuracy: %46.09
Epoch: 1, Steps: 1, Discriminator Accuracy: %54.30, GAN Accuracy: %0.00
Epoch: 2, Steps: 1, Discriminator Accuracy: %50.00, GAN Accuracy: %0.00
Epoch: 3, Steps: 1, Discriminator Accuracy: %50.00, GAN Accuracy: %0.00
Epoch: 4, Steps: 1, Discriminator Accuracy: %50.00, GAN Accuracy: %0.00
Epoch: 5, Steps: 1, Discriminator Accuracy: %50.00, GAN Accuracy: %0.00
Epoch: 6, Steps: 1, Discriminator Accuracy: %50.00, GAN Accuracy: %0.00
Epoch: 7, Steps: 1, Discriminator Accuracy: %50.00, GAN Accuracy: %0.00
Epoch: 8, Steps: 1, Discriminator Accuracy: %50.00, GAN Accuracy: %0.00
Epoch: 9, Steps: 1, Discriminator Accuracy: %50.00, GAN Accuracy: %0.00
Epoch: 10, Steps: 1, Discriminator Accuracy: %50.00, GAN Accuracy: %0.00
Epoch: 11, Steps: 1, Discriminator Accuracy: %50.00, GAN Accuracy: %0.00
Epoch: 12, Steps: 1, Discriminator Accuracy: %50.00, GAN Accuracy: %0.00
Epoch: 13, Steps: 1, Discriminator Accuracy: %50.00, GAN Acc

AssertionError: Nesting violated for default stack of <class 'tensorflow.python.client.session.Session'> objects

## Plot Loss

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_loss_g, label="Generator Loss");
plt.plot(train_loss_d, label="Discriminator Loss");
plt.legend();

## Plot Accuracy

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_acc_g, label="Generator Accuracy");
plt.plot(train_acc_d, label="Discriminator Accuracy");
plt.legend();